# Calling vald-client-clj APIs using Clojupyter

In this note, how to call vald-client-clj APIs using Clojupyter is described.

## Run the containers

First, run a Vald cluster or a Vald Agent NGT container.

- [Vald Get Started Document](https://vald.vdaas.org/docs/tutorial/get-started/)

In this example, a Vald Agent NGT container is used.
Please create a docker network to allow the containers to access by name.

    $ docker create network clojupyter-vald

To run a Vald Agent NGT, 

    $ docker run -v path-to-config-dir:/etc/server --network clojupyter-vald --rm --name vald-agent-ngt -it vdaas/vald-agent-ngt:nightly
    2020-06-29 06:12:59     [INFO]: maxprocs: Leaving GOMAXPROCS=3: CPU quota undefined
    2020-06-29 06:12:59     [INFO]: service agent ngt v0.0.0 starting...
    2020-06-29 06:12:59     [INFO]: daemon start
    2020-06-29 06:12:59     [INFO]: server pprof executing preStartFunc
    2020-06-29 06:12:59     [INFO]: server grpc executing preStartFunc
    2020-06-29 06:12:59     [INFO]: server prometheus executing preStartFunc
    2020-06-29 06:12:59     [INFO]: REST server pprof starting on 0.0.0.0:6060
    2020-06-29 06:12:59     [INFO]: gRPC server grpc starting on 0.0.0.0:8081
    2020-06-29 06:12:59     [INFO]: REST server readiness starting on 0.0.0.0:3001
    2020-06-29 06:12:59     [INFO]: REST server prometheus starting on 0.0.0.0:6061
    
Then, please start the clojupyter-vald-sample container.

    $ docker run -p 8888:8888 --network clojupyter-vald -it rinx/clojupyter-vald-sample

Please access to the http://localhost:8888 to open the Jupyter Notebook.

## Require vald-client-clj library and create a client

To require vald-client-clj library, please run the following:

In [10]:
(require '[vald-client-clj.core :as vald])

nil

and then, create a agent client.

In [11]:
(def client
  (vald/agent-client "vald-agent-ngt" 8081))

#'user/client

If you're using Vald cluster, please create a gateway client instead.

```clojure
(def client
  (vald/vald-client "domain of cluster" 8081))
```

## Call vald-client-clj APIs

You can use vald-client-clj APIs like the followings.


In [12]:
(def dimension 784)

#'user/dimension

In [13]:
;; insert
(-> client
    (vald/insert "meta" (take dimension (repeatedly rand))))

{}

In [14]:
;; remove-id
(-> client
    (vald/remove-id "meta"))

{}

In [15]:
;; stream insert
(-> client
    (vald/stream-insert
     identity
     (->> (take 1000 (range))
         (map (fn [i]
                  {:id (str "meta" i)
                   :vector (take dimension (repeatedly rand))}))
         (vec)))
    (deref))

{:status :done, :count 1000}

please wait for finishing creating index.

After index created, send search requests.

In [19]:
;; get-object
(-> client
    (vald/get-object "meta1"))

{:id "meta1", :vector (0.17312896 0.8915138 0.58590436 0.59255344 0.2863685 0.107937366 0.12550691 0.5917329 0.7696714 0.31319985 0.7738925 0.6885707 0.9783047 0.37719536 0.15314609 0.29829958 0.29871082 0.06326779 0.036677755 0.39726794 0.81643224 0.4840097 0.99045813 0.85241336 0.3008739 0.93103594 0.6570961 0.9254465 0.7929452 0.37868908 0.06889705 0.011956582 0.25838095 0.021205034 0.13092524 0.43834445 0.35254723 0.9186857 0.076222494 0.05843601 0.33179814 0.19929764 0.26272237 0.46682194 0.54195446 0.2934352 0.15502921 0.87741697 0.716236 0.36015645 0.714115 0.103426374 0.5229846 0.40702158 0.7041204 0.61885214 0.4177747 0.6015332 0.6178275 0.381868 0.8750141 0.9256976 0.02100685 0.9568365 0.5008978 0.6347987 0.120722614 0.48443726 0.36258343 0.94729275 0.27381232 0.21832038 0.1077154 0.014806556 0.9691418 0.6718955 0.093933456 0.4995394 0.54086757 0.90031046 0.799758 0.020939777 0.49982268 0.08037008 0.24625266 0.32434902 0.9062732 0.3450914 0.023122925 0.27137372 0.64967763 0.7

In [21]:
;; search
(-> client
    (vald/search {:num 3} (vec (take 784 (repeatedly rand))))
    (clojure.pprint/pprint))

[{:id "meta355", :distance 10.588243}
 {:id "meta354", :distance 10.793549}
 {:id "meta771", :distance 10.845719}]


nil

In [35]:
;; stream search
(def results (atom []))
(-> client
    (vald/stream-search
     (fn [res]
         (swap! results conj res))
     {:num 3}
     (vec (take 3 (repeatedly #(vec (take dimension (repeatedly rand)))))))
    (deref))
(clojure.pprint/pprint
 (deref results))

[[{:id "meta856", :distance 10.693958}
  {:id "meta442", :distance 10.771862}
  {:id "meta484", :distance 10.84279}]
 [{:id "meta616", :distance 10.6402}
  {:id "meta102", :distance 10.909838}
  {:id "meta582", :distance 10.945793}]
 [{:id "meta975", :distance 10.838391}
  {:id "meta619", :distance 10.841153}
  {:id "meta737", :distance 10.9196}]]


nil

In [36]:
;; stream search-by-id
(def results2 (atom []))
(-> client
    (vald/stream-search-by-id
     (fn [res]
         (swap! results2 conj res))
     {:num 3}
     ["meta1" "meta2" "meta3"])
    (deref))
(clojure.pprint/pprint
 (deref results2))

[[{:id "meta143", :distance 10.825673}
  {:id "meta42", :distance 11.002299}
  {:id "meta711", :distance 11.005922}]
 [{:id "meta2", :distance 0.0}
  {:id "meta721", :distance 10.6902}
  {:id "meta612", :distance 10.70013}]
 [{:id "meta3", :distance 0.0}
  {:id "meta561", :distance 10.533728}
  {:id "meta301", :distance 10.557975}]]


nil

In [37]:
(vald/close client)

#object[io.grpc.internal.ManagedChannelImpl 0x28f22d67 "ManagedChannelImpl{logId=5, target=vald-agent-ngt:8081}"]